In [1]:
# Importing the libraries
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
# Import dataset
main_dataset = pd.read_csv("../processed_files/main_dataset.csv", index_col=[0])

#Without speed,shape_dist_traveled,ritardo_partenza,oat,humidity,t apparente, wind dicetion,rain , snowdepth
main_dataset = main_dataset.drop(['arrival_time','departure_time','codice_stazione','stazione','codice','orario_arrivo','orario_partenza',
                                  'data_giorno','trip_id','time','orario','stop_lat','stop_lon','location_id','shape_dist_traveled',
                                  'wmo_code','relative_humidity_2m (%)','snow_depth',
                                  'wind_direction_10m (°)', 'opt','oat','app_temp','wind_speed','rain'
                                 ], axis=1)

In [3]:
main_dataset.head(1)


,stop_sequence,time_diff,speed,dist_diff,stop_code,ritardo_arrivo,ritardo_partenza,temperature,snow_fall
0,1,0.0,0.0,0.0,6404,0,3,10.0,0.0


In [4]:
main_dataset.shape

(168205, 9)

In [5]:
for index,row in main_dataset.iterrows():
    if row['ritardo_partenza'] < 0:
        main_dataset.loc[index,'ritardo_partenza'] = 0
    if row['ritardo_arrivo'] < 0:
        main_dataset.loc[index,'ritardo_arrivo'] = 0

In [6]:
train, test = train_test_split(
    main_dataset, test_size=0.2, random_state=42, shuffle=True)

In [7]:
train

,stop_sequence,time_diff,speed,dist_diff,stop_code,ritardo_arrivo,ritardo_partenza,temperature,snow_fall
53879,2,0.08,26.69,2.22,6501,0,0,15.0,0.0
103917,9,0.07,57.58,3.84,6306,6,6,18.5,0.0
123196,12,0.13,36.69,4.89,6300,0,1,12.2,0.0
23578,2,0.10,80.19,8.02,6402,1,1,31.7,0.0
160013,3,0.05,52.17,2.61,6223,2,2,11.0,0.0
...,...,...,...,...,...,...,...,...,...
119995,10,0.07,42.37,2.82,6302,9,9,18.1,0.0
103767,9,0.07,57.58,3.84,6306,7,7,24.3,0.0
132106,13,0.03,43.54,1.45,6323,2,2,9.9,0.0
147124,6,0.15,46.16,6.92,6227,3,4,9.9,0.0


In [ ]:
# Defining the model
model = keras.Sequential([
    keras.layers.Dense(16, input_shape=(7,), activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(200, activation='softmax')
])
 
# Compiling the model
model.compile(optimizer='adam',
                     loss=keras.losses.SparseCategoricalCrossentropy(),
                     metrics=['accuracy'])
 
# Constructing the input
x = np.column_stack((train.stop_sequence.values, train.time_diff.values,
                    train.speed.values, train.dist_diff.values, train.stop_code.values,
                    train.temperature.values, train.ritardo_partenza.values)) 
y = train.ritardo_arrivo.values
 
# fitting the model
model.fit(x, y, epochs=100, batch_size=2)

Epoch 1/100
  115/67282 [..............................] - ETA: 29s - loss: 129.6877 - accuracy: 0.1304        

2023-11-08 17:14:40.199466: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


67282/67282 [==============================] - 28s 407us/step - loss: 2.6579 - accuracy: 0.3096
Epoch 2/100
67282/67282 [==============================] - 27s 408us/step - loss: 2.1187 - accuracy: 0.3179
Epoch 3/100
67282/67282 [==============================] - 28s 412us/step - loss: 2.1209 - accuracy: 0.3179
Epoch 4/100
67282/67282 [==============================] - 28s 416us/step - loss: 2.1213 - accuracy: 0.3179
Epoch 5/100
67282/67282 [==============================] - 28s 411us/step - loss: 2.1220 - accuracy: 0.3179
Epoch 6/100
67282/67282 [==============================] - 28s 416us/step - loss: 2.1230 - accuracy: 0.3179
Epoch 7/100
67282/67282 [==============================] - 28s 411us/step - loss: 2.1227 - accuracy: 0.3179
Epoch 8/100
67282/67282 [==============================] - 28s 409us/step - loss: 2.1230 - accuracy: 0.3179
Epoch 9/100
67282/67282 [==============================] - 28s 412us/step - loss: 2.1233 - accuracy: 0.3179
Epoch 10/100
67282/67282 [==============

In [ ]:
# Evaluating the model
x = np.column_stack((test.stop_sequence.values, test.time_diff.values,
                    test.speed.values, test.dist_diff.values, test.stop_code.values,
                    test.temperature.values, test.app_temp.values, test.rain.values,
                    test.snow_fall.values, test.wind_speed.values)) 
y = test.ritardo_arrivo.values

model.evaluate(x, y, batch_size=8)